# Generate rxns that only you need

Han, Kehang (hkh12@mit.edu)

## Set-up

In [1]:
import os
from rmgpy.rmg.main import RMG, CoreEdgeReactionModel
from rmgpy.data.rmg import RMGDatabase
from rmgpy import settings

In [2]:
# set-up RMG object
rmg = RMG()
rmg.reactionModel = CoreEdgeReactionModel()

# load kinetic database and forbidden structures
rmg.database = RMGDatabase()
path = os.path.join(settings['database.directory'])

In [3]:
# set up kinetics db
from rmgpy.data.rmg import database
# forbidden structure loading
database.loadForbiddenStructures(os.path.join(path, 'forbiddenStructures.py'))
# kinetics family Disproportionation loading
database.loadKinetics(os.path.join(path, 'kinetics'), kineticsFamilies='default')
# set rmg kinetics estimator
rmg.reactionModel.kineticsEstimator = "rate rules"

/home/kehang/Code/rmgmaster/RMG-Py/rmgpy/scoop_framework/util.py:77: RuntimeWarning: SCOOP was not started properly.
Be sure to start your program with the '-m scoop' parameter. You can find further information in the documentation.

  RuntimeWarning


In [4]:
# set up thermo and transport db
database.loadThermo(os.path.join(path, 'thermo'))
database.loadTransport(os.path.join(path, 'transport'))

In [5]:
# set up chemkin file saving condition
from rmgpy.chemkin import ChemkinWriter

output_path = os.path.abspath('./') + '/rxn_gen/'
rmg.outputDirectory = output_path
rmg.saveEdgeSpecies = True
rmg.attach(ChemkinWriter(output_path))

## Input

In [6]:
from reactionGenerator import loadInput, pathwayGenerator
pathway_inputfile = 'rxn_gen/input.dat'
family_list, reactants_list, products_list = loadInput(pathway_inputfile)

In [7]:
# create input species and family of interest
from rmgpy.rmg.model import Species
for family, reactants_str, products_str in zip(family_list, reactants_list, products_list):
    reactants = []
    products = []
    for reactant_str in reactants_str:
        reactants.append(Species().fromSMILES(reactant_str))
    for product_str in products_str:
        products.append(Species().fromSMILES(product_str))

    # gen thermo and transport for reactants
    for spec in reactants:
        spec.generateThermoData(database)
        spec.generateTransportData(database)
    # pathway generation
    pathwayGenerator(reactants, products, rmg, family)

## Output

In [8]:
# save chemkin file for this step
rmg.reactionModel.outputSpeciesList.extend(rmg.reactionModel.edge.species)
rmg.reactionModel.outputReactionList.extend(rmg.reactionModel.edge.reactions)
rmg.saveEverything()